In [ ]:
import numpy as np

import torch
from torch_geometric.data import Data

import import_ipynb
from gnn import GCN
from constants import *

In [ ]:
def adj_info(adj):
    n = len(adj)
    m = 0
    for u in range(n): m+=len(adj[u])
    return n,m

In [ ]:
def txt2adj(graph_name):
    if '.txt' not in graph_name: graph_name+='.txt'
    with open(GRAPH_DIR+graph_name,'r') as f:
        n,m = map(int,f.readline().split())
        adj = [[] for i in range(n)]
        for _ in range(m):
            u,v,p = f.readline().split()
            u,v,p = int(u),int(v),float(p)
            adj[u].append((v,p))
    return n,m,adj

def txt2coo(graph_name):
    if '.txt' not in graph_name: graph_name+='.txt'
    edge_index = [[],[]]
    edge_attr = []
    with open(GRAPH_DIR+graph_name,'r') as f:
        n,m = map(int,f.readline().split())
        adj = [[] for i in range(n)]
        for _ in range(m):
            u,v,p = f.readline().split()
            u,v,p = int(u),int(v),float(p)
            edge_index[0].append(u)
            edge_index[1].append(v)
            edge_attr.append([p])
    edge_index = torch.tensor(edge_index)
    edge_attr = torch.tensor(edge_attr)
    return edge_index, edge_attr

def adj2coo(adj_list):
    edge_index = [[],[]]
    edge_attr = []
    n = len(adj_list)
    for u in range(n):
        for v,p in adj_list[u]:
            edge_index[0].append(u)
            edge_index[1].append(v)
            edge_attr.append([p])
    edge_index = torch.tensor(edge_index)
    edge_attr = torch.tensor(edge_attr)
    return edge_index, edge_attr

def adj2Data(adj_list, seed_idx, y=None, prob=None):
    n = len(adj_list)
    is_seed = np.zeros(n, dtype=int)
    is_seed[seed_idx] = 1
    edge_index, edge_attr = adj2coo(adj_list)
    seed = torch.from_numpy(np.expand_dims(is_seed,axis=-1)).float()
    if prob is not None: y = torch.from_numpy(np.expand_dims(prob,axis=-1)).float()
    data = Data(x=seed, edge_index=edge_index, edge_attr=edge_attr, y=y)
    return data

def adj2edges(adj_list):
    n = len(adj_list)
    edges = []
    for u in range(n):
        for v,p in adj_list[u]: edges.append((u,v,p))
    return np.array(edges)

def list2mat(adj_list):
    n = len(adj_list)
    adj_mat = [[0]*n for _ in range(n)]
    for u in range(n):
        for v,p in adj_list[u]: adj_mat[u][v] = p
    return adj_mat

def mat2list(adj_mat):
    n = len(adj_mat)
    adj_mat = np.array(adj_mat, dtype=np.float32)
    adj_list = [[] for _ in range(n)]
    for u in range(n):
        for v in np.where(adj_mat[u,:]>0)[0]: adj_list[u].append([v,adj_mat[u][v]])
    return adj_list

In [ ]:
def set_gpu(gpu_num='cpu'):
    if gpu_num=='cpu': device='cpu'
    else:
        device = torch.device(f"cuda:{gpu_num}" if torch.cuda.is_available() else "cpu")
        torch.cuda.set_device(device)
    return device

def load_model(model_name, device='cpu', **kwargs):
    model = GCN(**kwargs)
    if model_name: model.load_state_dict(torch.load(MODEL_DIR+model_name, map_location=device))
    model.to(device)
    return model